# Design a prototype

In [2]:
import numpy as np 
class AR:
    def __init__(self, *phi, epsilon, Y0):
        self.phi=np.array(phi)
        self.epsilon=epsilon
        self.Y0=np.array(Y0)
        self.memory=np.array([])
        self.YPast=np.array(self.Y0)

An instance

In [3]:
import numpy as np

def Epsilon(mu, sigma):
    def draw(size):
        return np.random.normal(mu, sigma, size)
    return draw

epsilon=Epsilon(0, 0.4) 

In [7]:
epsilon(1), epsilon(10)

(array([0.43632337]),
 array([-0.10602412,  0.00938788, -1.11997556, -0.29386045, -0.15305376,
         0.34173417, -0.70611284, -0.25917241,  0.01452547, -0.33869137]))

In [8]:
ar = AR(0.8, -0.35, epsilon=epsilon, Y0=[0,0])

In [12]:
# an instance of the basic prototype of a class
ar.phi, ar.epsilon(10), ar.Y0, ar.YPast

(array([ 0.8 , -0.35]),
 array([ 0.10330481,  0.58060874, -1.06616693,  0.08490227,  0.09856198,
         0.03261635, -0.26732027,  0.13959167, -0.67900763,  0.52328218]),
 array([0, 0]),
 array([0, 0]))

# Packing

In [13]:
# Packing is to pack all non-key-value-pair argument values into a tuple
def test(*args, epsilon):
    print(args)
    print(epsilon)

test(1,2,3, epsilon="Hi")

(1, 2, 3)
Hi


Exercise CES and packing

In [30]:
def CESUtility(*alpha, rho):
    def function(*X):
        X=np.array(X)
        result=alpha*X**rho
        return result.sum()**(1/rho)
    return function

# example 1
u = CESUtility(0.3, 0.3, 0.4, rho=2)
u(2, 4, 5)

4.0

In [31]:
# example 2
u2 = CESUtility(0.2, 0.2, 0.3, 0.3, rho=2)
u2(1, 2, 2, 1)

1.5811388300841898

# Memory

ar.simulate_nPeriods(10) will simulate 10 periods of the AR(k) model.

Algorithm:

0. Given eps = ar.epsilon(1)
1. Compute Y_one_step_ahead using ar.phi and ar.YPast, and ar.epsilon.
   * ar.phi@ar.YPast + eps
2. Update YPast by appending Y_one_step_ahead to YPast front and removing the last element of YPast.
3. Update ar.memroy to remember Y_one_step_ahead.

In [34]:
eps=ar.epsilon(1)

# Simulate one period of ar process
y_onePeriod_ahead=ar.phi@ar.YPast+eps

# Update YPast
ar.YPast=np.append(y_onePeriod_ahead, ar.YPast[:-1])

# append the new element to the memory
ar.memory=np.append(ar.memory, y_onePeriod_ahead)

In [36]:
ar.YPast

array([0.56815207, 0.        ])

In [7]:
def simulate_onePeriod(ar, eps):
    # Simulate one period of ar process
    y_onePeriod_ahead=ar.phi@ar.YPast+eps
    
    # Update YPast
    ar.YPast=np.append(y_onePeriod_ahead, ar.YPast[:-1])

    # append the new element to the memory
    ar.memory=np.append(ar.memory, y_onePeriod_ahead)

Test your helper function

In [8]:
ar=AR(0.8,-0.35,epsilon=epsilon,Y0=[0,0])
ar.memory

array([], dtype=float64)

In [9]:
simulate_onePeriod(ar, ar.epsilon(1))
ar.memory

array([-0.23302462])

In [10]:
# generate n periods of ar process
n=10
eps=ar.epsilon(n)
for i in range(n):
    simulate_onePeriod(ar, eps[i])

In [11]:
ar.memory

array([-0.23302462,  0.018891  ,  0.30617862,  0.99170742,  0.58824097,
       -0.36191181,  0.06524614,  0.42434141,  0.62185992,  0.22908233,
       -0.03395584])

In [14]:
import matplotlib.pyplot

Helper functions

In [ ]:
def simulate_onePeriod(ar, eps):
    # Simulate one period of ar process
    y_onePeriod_ahead = ar.phi@ar.YPast + eps

    # Update YPast
    ar.YPast = np.append(y_onePeriod_ahead, ar.YPast[:-1])

    ## append the new element to the memory
    ar.memory = np.append(ar.memory, y_onePeriod_ahead)

def simulate_nPeriods(ar, n):
    eps = ar.epsilon(n)
    for i in range(n):
        simulate_onePeriod(ar)

For loop

In [16]:
# Range
a = range(10)
iter_a=a.__iter__() # obtain the iterator of a

In [22]:
iter_a.__next__()

5

In [24]:
# List
a=["apple", "banana", "orange"]
iter_a=a.__iter__() # obtain the iterator of a

In [25]:
iter_a.__next__()

'apple'

In [26]:
for i in a:
    print(i)

apple
banana
orange


In [41]:
# Dictionary
a={"name":"John", "age":36, "country":"Norway"}
iter_a=a.__iter__() # obtain the iterator of a

In [42]:
iter_a.__next__()

'name'

In [47]:
for key in a:
    print(a[key])

John
36
Norway


In [50]:
a_items=a.items()
a_items_iter=a_items.__iter__()

In [51]:
a_items_iter.__next__()

('name', 'John')

In [53]:
for key, value in a.items():
    print(f'{key} is {value}')

name is John
age is 36
country is Norway


unpacking

In [54]:
k,v=("name","John")
print(k)
print(v)

name
John


Modulize your code

In [1]:
# impoer AR module
import sys
sys.path.append("py/AR/univariate.py")
from AR import AR
import numpy as np

def Epsilon(mu, sigma):
    def draw(size):
        return np.random.normal(mu,sigma,size)
    return draw

epsilon = Epsilon(0,0.4)

In [3]:
"without seed", epsilon(1), epsilon(10)

('without seed',
 array([0.50461103]),
 array([ 0.41589611,  0.43368574, -0.53443847,  0.51794254,  0.35120752,
        -0.20817183, -0.07233734, -0.36763952,  0.9234069 ,  1.05237105]))

In [2]:
np.random.seed(2384)
"with seed", epsilon(1), epsilon(10)

('with seed',
 array([-0.27316285]),
 array([ 0.2479762 , -0.1758513 , -0.37261468, -0.06035985,  0.09375106,
        -0.40309579, -0.29947629, -0.58143623, -0.30846613, -0.2555456 ]))

In most application , you should set seed for reproducible results.

In [3]:
ar = AR(0.8, -0.35, epsilon=epsilon, Y0=[0,0])

# without seed
ar.simulate_nPeriods(10)
ar.memory

array([ 0.06952085,  0.41455001,  0.22252219,  0.52004695,  0.53057793,
        1.77308651,  1.51729585,  0.75829884, -0.33874324, -0.34056626])